In [1]:
# 1. 请先使用上位机升级机器人电机通信固件版本到0.3.12.25 以上，驱动固件版本升级到0.2.10.26 以上 详细使用请参考上位机使用说明
# 2. 请使用上位机对机器人零位进行标定（无绝编则不用）
# 3. 请使用上位机调整机器人GR1T2电机pd参数（注意不是pid参数），
#    参考值：上肢电机10-14/30-34电机 p：200，d：15;
#           上肢电机15-16/35-36电机 p：100，d:10;
#           腰部电机90              p:300，d:18;
#           腰部电机91-92            p:500, d:20。
#           头部电机93-95            p:40, d:0
#           腿部电机50-53/70-73电机 p:100,d:10;
#           脚踝电机54-55/74-75电机 p:100,d:10;
#    可以根据软硬自己调节,腕关节可以小一点
# 4. 请使用指定的docker镜像进行开发，镜像名为fourier_hardware:latest，请使用开源仓库中的docker启动脚本启动docker
# 5. 注意 t2 头部请在上电之前放在中心位置

In [2]:
# fourier_hardware_py 用于对机器人进行分组控制
import fourier_hardware_py
# grx_sot_py 用于对机器人进行运动学，动力学求解
import grx_sot_py
import numpy as np
from dataclasses import dataclass, field
import pickle
import time
import math

In [3]:
# 加载对应机型的config配置文件
config = fourier_hardware_py.HardwareConfigHelper("/usr/local/config/fourier_hardware/gr1t2")

In [4]:
# 配置各控制组的控制模式为PD模式，并忽略下肢控制的运动
config.changeControlGroupMotorMode("left_manipulator","pd")
config.changeControlGroupMotorMode("right_manipulator","pd")
config.changeControlGroupMotorMode("left_wrist","pd")
config.changeControlGroupMotorMode("right_wrist","pd")
config.changeControlGroupMotorMode("head", "pd")
config.changeControlGroupMotorMode("head_direct", "pd")
config.changeControlGroupMotorMode("waist","pd")
config.changeControlGroupMotorMode("left_leg","pd")
config.changeControlGroupMotorMode("right_leg","pd")
config.changeControlGroupMotorMode("left_ankle","pd")
config.changeControlGroupMotorMode("right_ankle","pd")

True

In [5]:
hardware = fourier_hardware_py.FourierHardware("/usr/local/config/fourier_hardware/gr1t2",config)

FSA(192.168.137.93): FSA subscribe port 2336 is connected to local port: 47149
FSA(192.168.137.94): FSA subscribe port 2336 is connected to local port: 35752
FSA(192.168.137.95): FSA subscribe port 2336 is connected to local port: 50820
FSA(192.168.137.74): FSA subscribe port 2336 is connected to local port: 50601
FSA(192.168.137.75): FSA subscribe port 2336 is connected to local port: 42938
FSA(192.168.137.70): FSA subscribe port 2336 is connected to local port: 38764
FSA(192.168.137.71): FSA subscribe port 2336 is connected to local port: 37348
FSA(192.168.137.72): FSA subscribe port 2336 is connected to local port: 36906
FSA(192.168.137.73): FSA subscribe port 2336 is connected to local port: 40026
FSA(192.168.137.10): FSA subscribe port 2336 is connected to local port: 57199
FSA(192.168.137.11): FSA subscribe port 2336 is connected to local port: 54077
FSA(192.168.137.12): FSA subscribe port 2336 is connected to local port: 36577
FSA(192.168.137.13): FSA subscribe port 2336 is conn

In [6]:
robot = grx_sot_py.RobotWrapper("/usr/local/config/fourier_hardware/gr1t2", grx_sot_py.Flags.ignore_collisions)
# 定义机器人对应的控制组和与控制组匹配的关节顺序，输出对应的字典信息
group_names = []
group_joints = []
group_joints_pos_order = {}
group_joints_vel_order = {}

for group_name in config.getControlGroupNames():
    if config.getControlGroupMotorMode(group_name)== "ignore":
        continue
    group_names.append(group_name)
    robot_joint_names = hardware.getControlGroupJointNames(group_name)
    group_joints.append(robot_joint_names)
    joint_pos_orders = []
    joint_vel_orders = []
    for robot_joint_name in robot_joint_names:
        joint_pos_orders.append(robot.get_joint_offset(robot_joint_name))
        joint_vel_orders.append(robot.get_joint_v_offset(robot_joint_name))
    group_joints_pos_order[group_name]= joint_pos_orders
    group_joints_vel_order[group_name] = joint_vel_orders
print(config.getRobotName())
print(config.getControlGroupNames())
print(group_joints_pos_order)
print(group_joints_vel_order)

gr1t2
['head', 'head_direct', 'waist', 'left_manipulator', 'right_manipulator', 'left_wrist', 'right_wrist', 'left_leg', 'right_leg', 'left_ankle', 'right_ankle']
{'head': [22, 23], 'head_direct': [24], 'waist': [19, 20, 21], 'left_manipulator': [25, 26, 27, 28, 29], 'right_manipulator': [32, 33, 34, 35, 36], 'left_wrist': [30, 31], 'right_wrist': [37, 38], 'left_leg': [7, 8, 9, 10], 'right_leg': [13, 14, 15, 16], 'left_ankle': [11, 12], 'right_ankle': [17, 18]}
{'head': [21, 22], 'head_direct': [23], 'waist': [18, 19, 20], 'left_manipulator': [24, 25, 26, 27, 28], 'right_manipulator': [31, 32, 33, 34, 35], 'left_wrist': [29, 30], 'right_wrist': [36, 37], 'left_leg': [6, 7, 8, 9], 'right_leg': [12, 13, 14, 15], 'left_ankle': [10, 11], 'right_ankle': [16, 17]}
[2025-05-20 19:31:35] [9873] [error] 192.168.137.32 Get subscribe data failed with code: -302


## 对head control group 进行测试

In [7]:
hardware.setControlGroupPIDParams("head",[40,40],[0,0],[0,0])

True

In [8]:
hardware.enableControlGroup("head")

[2025-05-20 19:31:38] [9873] [error] 192.168.137.93 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.94 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.74 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.75 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.70 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.71 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.10 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.11 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.12 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.13 Get subscribe data failed with code: -302
[2025-05-20 19:31:38] [9873] [error] 192.168.137.14 Get subs

True

In [9]:
# 生成机器人头部关节从标定位置到urdf零位的轨迹
head_point_init = [-0.0, 0.]
head_state = hardware.getControlGroupState("head")
head_traj = fourier_hardware_py.bridgeTrajectory(head_state.q[-2:], head_point_init)
# 驱动头部控制组关节回到urdf零位
for i in range(len(head_traj[1])):
    start_time = time.perf_counter()
    hardware.setControlGroupPosCmd("head",head_traj[1][i])
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/500 :
        time.sleep(0.002-execution_time)
    else:
        print("control loop is slow",execution_time)

## 对head_direct control group 进行测试

In [10]:
hardware.setControlGroupPIDParams("head_direct",[40],[0],[0])

True

In [11]:
hardware.enableControlGroup("head_direct")

True

In [12]:
head_direct_point_init = [0.0]
head_direct_state = hardware.getControlGroupState("head_direct")
head_direct_traj = fourier_hardware_py.bridgeTrajectory(head_direct_state.q[-1:], head_direct_point_init)
for i in range(len(head_direct_traj[1])):
    start_time = time.perf_counter()
    hardware.setControlGroupPosCmd("head_direct",head_direct_traj[1][i])
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/500 :
        time.sleep(0.002-execution_time)
    else:
        print("control loop is slow",execution_time)

In [13]:
hardware.setControlGroupPIDParams("left_wrist",[40,40],[0,0],[0,0])

True

## 对left wrist control group 进行测试

In [14]:
hardware.enableControlGroup("left_wrist")

[2025-05-20 19:31:48] [9873] [error] 192.168.137.16 Get subscribe data failed with code: -302


True

[2025-05-20 19:31:48] [9873] [error] 192.168.137.50 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.51 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.52 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.53 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.30 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.31 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.32 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.33 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.34 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.35 Get subscribe data failed with code: -302
[2025-05-20 19:31:48] [9873] [error] 192.168.137.36 Get subs

In [15]:
left_wrist_point_init = [-0.0, 0.0]
left_wrist_state = hardware.getControlGroupState("left_wrist")
left_wrist_traj = fourier_hardware_py.bridgeTrajectory(left_wrist_state.q[-2:], left_wrist_point_init)
for i in range(len(left_wrist_traj[1])):
    start_time = time.perf_counter()
    hardware.setControlGroupPosCmd("left_wrist",left_wrist_traj[1][i])
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/500 :
        time.sleep(0.002-execution_time)
    else:
        print("control loop is slow",execution_time)

In [16]:
hardware.setControlGroupPIDParams("right_wrist",[40,40],[0,0],[0,0])

True

## 对right wrist control group 进行测试

In [17]:
hardware.enableControlGroup("right_wrist")

[2025-05-20 19:31:52] [9873] [error] 192.168.137.36 Get subscribe data failed with code: -302
[2025-05-20 19:31:52] [9873] [error] 192.168.137.90 Get subscribe data failed with code: -302


True

[2025-05-20 19:31:52] [9873] [error] 192.168.137.92 Get subscribe data failed with code: -302


In [18]:
right_wrist_point_init = [-0.0, 0.0]
right_wrist_state = hardware.getControlGroupState("right_wrist")
right_wrist_traj = fourier_hardware_py.bridgeTrajectory(right_wrist_state.q[-2:], right_wrist_point_init)
for i in range(len(right_wrist_traj[1])):
    start_time = time.perf_counter()
    hardware.setControlGroupPosCmd("right_wrist",right_wrist_traj[1][i])
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/500 :
        time.sleep(0.002-execution_time)
    else:
        print("control loop is slow",execution_time)

## 对 left_ankle control group 进行测试

In [19]:
hardware.setControlGroupPIDParams("left_ankle",[40,40],[0,0],[0,0])

True

In [20]:
hardware.enableControlGroup("left_ankle")

[2025-05-20 19:31:55] [9873] [error] 192.168.137.70 Get subscribe data failed with code: -302


True

[2025-05-20 19:31:55] [9873] [error] 192.168.137.73 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.10 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.11 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.13 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.15 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.16 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.55 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.50 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.52 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.31 Get subscribe data failed with code: -302
[2025-05-20 19:31:55] [9873] [error] 192.168.137.32 Get subs

In [21]:
left_ankle_point_init = [-0.0, 0.0]
left_ankle_state = hardware.getControlGroupState("left_ankle")
left_ankle_traj = fourier_hardware_py.bridgeTrajectory(left_ankle_state.q[-2:], left_ankle_point_init)
for i in range(len(left_ankle_traj[1])):
    start_time = time.perf_counter()
    hardware.setControlGroupPosCmd("left_ankle",left_ankle_traj[1][i])
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/500 :
        time.sleep(0.002-execution_time)
    else:
        print("control loop is slow",execution_time)

## 对 right_ankle control group 进行测试

In [22]:
hardware.setControlGroupPIDParams("right_ankle",[40,40],[0,0],[0,0])

True

In [23]:
hardware.enableControlGroup("right_ankle")

[2025-05-20 19:31:58] [9873] [error] 192.168.137.54 Get subscribe data failed with code: -302


True

[2025-05-20 19:31:58] [9873] [error] 192.168.137.51 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.52 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.53 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.30 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.31 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.32 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.33 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.34 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.36 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.90 Get subscribe data failed with code: -302
[2025-05-20 19:31:58] [9873] [error] 192.168.137.91 Get subs

In [24]:
right_ankle_point_init = [-0.0, 0.0]
right_ankle_state = hardware.getControlGroupState("right_ankle")
right_ankle_traj = fourier_hardware_py.bridgeTrajectory(right_ankle_state.q[-2:], right_ankle_point_init)
for i in range(len(right_ankle_traj[1])):
    start_time = time.perf_counter()
    hardware.setControlGroupPosCmd("right_ankle",right_ankle_traj[1][i])
    end_time = time.perf_counter()
    #计算控制循环的周期，保持控制频率的稳定
    execution_time = (end_time - start_time)
    if  execution_time < 1.0/500 :
        time.sleep(0.002-execution_time)
    else:
        print("control loop is slow",execution_time)

In [25]:
hardware.disableRobot()

[2025-05-20 19:32:00] [9873] [error] 192.168.137.70 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.71 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.72 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.73 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.16 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.54 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.50 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.51 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.52 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.53 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.30 Get subs

True

Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.30 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.31 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.32 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.33 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.34 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.36 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.90 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.91 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.92 Get subscribe data failed with code: -302
[2025-05-20 19:32:00] [9873] [error] 192.168.137.36 Get subscribe data failed with code: -302
[2025-05-20 19:32:

In [26]:
hardware.enableRobot()

[2025-05-20 19:32:05] [9873] [error] 192.168.137.93 Get subscribe data failed with code: -302


True

[2025-05-20 19:32:05] [9873] [error] 192.168.137.94 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.74 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.70 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.71 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.10 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.11 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.12 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.13 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.14 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.15 Get subscribe data failed with code: -302
[2025-05-20 19:32:05] [9873] [error] 192.168.137.54 Get subs

In [ ]:
hardware.disableRobot()

# 测试gr1t2绝对值编码器

In [5]:
# 计算各个输入的关节位置信息的误差
@dataclass
class PositionErrorMetrics:
    errors: np.ndarray = field(default_factory=lambda: np.array([]))
    rmse: float = 0.0
    max_error: float = 0.0
    mean_error: float = 0.0
    std_deviation: float = 0.0

    def calculate_metrics(self):
        if self.errors.size == 0:
            raise ValueError("Error array is empty, cannot calculate metrics.")

        # 均方根误差 (RMSE)
        self.rmse = np.sqrt(np.mean(np.square(self.errors)))

        # 最大误差 (Max Error)
        self.max_error = np.max(np.abs(self.errors))

        # 平均误差 (Mean Error)
        self.mean_error = np.mean(np.abs(self.errors))

    def display_metrics(self):
        print(f"RMSE: {self.rmse}")
        print(f"Max Error: {self.max_error}")
        print(f"Mean Error: {self.mean_error}")


In [6]:
# 此处会尝试与机器人建立连接，如果失败（存在control group无法连接），将会持续报错，请先检查机器人关节ip是否可以ping通，
# 如果不能ping通，请尝试重启机器人或寻求硬件支持
hardware = fourier_hardware_py.FourierHardware("/usr/local/config/fourier_hardware/gr1t2",config)
# 核心类，用于对gr1t2 绝编进行标定以及计算offset (gr1t2的绝编设置特殊，不同于gr2t2 和 grmini）
# 请程序每次关闭之后对机器人进行断电（！！！！！非常重要）
t2_abs_helper = fourier_hardware_py.T2ABSHelper("/usr/local/config/fourier_hardware/gr1t2")

FSA(192.168.137.93): FSA subscribe port 2336 is connected to local port: 42456
FSA(192.168.137.94): FSA subscribe port 2336 is connected to local port: 41815
FSA(192.168.137.95): FSA subscribe port 2336 is connected to local port: 47258
FSA(192.168.137.74): FSA subscribe port 2336 is connected to local port: 55578
FSA(192.168.137.75): FSA subscribe port 2336 is connected to local port: 45975
FSA(192.168.137.70): FSA subscribe port 2336 is connected to local port: 54576
FSA(192.168.137.71): FSA subscribe port 2336 is connected to local port: 40068
FSA(192.168.137.72): FSA subscribe port 2336 is connected to local port: 41295
FSA(192.168.137.73): FSA subscribe port 2336 is connected to local port: 37773
FSA(192.168.137.10): FSA subscribe port 2336 is connected to local port: 42420
FSA(192.168.137.11): FSA subscribe port 2336 is connected to local port: 52905
FSA(192.168.137.12): FSA subscribe port 2336 is connected to local port: 37507
FSA(192.168.137.13): FSA subscribe port 2336 is conn

In [7]:
# 根据配置文件定义对应的机器人类，支持运动学、动力学求解
robot = grx_sot_py.RobotWrapper("/usr/local/config/fourier_hardware/gr1t2", grx_sot_py.Flags.ignore_collisions)

In [8]:
# 定义机器人对应的控制组和与控制组匹配的关节顺序，输出对应的字典信息
group_names = []
group_joints = []
group_joints_pos_order = {}
group_joints_vel_order = {}

for group_name in config.getControlGroupNames():
    if config.getControlGroupMotorMode(group_name)== "ignore":
        continue
    group_names.append(group_name)
    robot_joint_names = hardware.getControlGroupJointNames(group_name)
    group_joints.append(robot_joint_names)
    joint_pos_orders = []
    joint_vel_orders = []
    for robot_joint_name in robot_joint_names:
        joint_pos_orders.append(robot.get_joint_offset(robot_joint_name))
        joint_vel_orders.append(robot.get_joint_v_offset(robot_joint_name))
    group_joints_pos_order[group_name]= joint_pos_orders
    group_joints_vel_order[group_name] = joint_vel_orders

In [9]:
print(config.getRobotName())
print(config.getControlGroupNames())
print(group_joints_pos_order)
print(group_joints_vel_order)
# 获取当前绝编位置，最多尝试3次，返回一个pair，代表成功与否，成功的话后面数组代表当前的绝编位置。
print(t2_abs_helper.getControlGroupABSAngle("waist",3))
print(t2_abs_helper.getControlGroupABSAngle("left_leg",3))
print(t2_abs_helper.getControlGroupABSAngle("right_leg",3))
print(t2_abs_helper.getControlGroupABSAngle("left_ankle",3))
print(t2_abs_helper.getControlGroupABSAngle("right_ankle",3))

gr1t2
['head', 'head_direct', 'waist', 'left_manipulator', 'right_manipulator', 'left_wrist', 'right_wrist', 'left_leg', 'right_leg', 'left_ankle', 'right_ankle']
{'head': [22, 23], 'head_direct': [24], 'waist': [19, 20, 21], 'left_manipulator': [25, 26, 27, 28, 29], 'right_manipulator': [32, 33, 34, 35, 36], 'left_wrist': [30, 31], 'right_wrist': [37, 38], 'left_leg': [7, 8, 9, 10], 'right_leg': [13, 14, 15, 16], 'left_ankle': [11, 12], 'right_ankle': [17, 18]}
{'head': [21, 22], 'head_direct': [23], 'waist': [18, 19, 20], 'left_manipulator': [24, 25, 26, 27, 28], 'right_manipulator': [31, 32, 33, 34, 35], 'left_wrist': [29, 30], 'right_wrist': [36, 37], 'left_leg': [6, 7, 8, 9], 'right_leg': [12, 13, 14, 15], 'left_ankle': [10, 11], 'right_ankle': [16, 17]}
(True, array([3.26316071, 0.58828163, 3.9730103 ]))
(True, array([3.08636928, 1.42545164, 3.19950032, 1.65900028]))
(True, array([1.67970896, 6.05730677, 0.54571366, 4.87920952]))
(True, array([1.78670406, 1.26284969]))
(True, arr

In [10]:
# 使能整个机器人
hardware.enableRobot()

[2025-05-20 19:37:25] [10093] [error] 192.168.137.95 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.70 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.72 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.11 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.13 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.15 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.16 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.54 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.55 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.50 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.

True

30 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.31 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.34 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.36 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.90 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.91 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.92 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.74 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.70 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.72 Get subscribe data failed with code: -302
[2025-05-20 19:37:25] [10093] [error] 192.168.137.11 Get subscribe data failed with code: -302
[2025

In [11]:
# 请注意在电机上电时，请确保头部处于中间位置！！！
print(hardware.getControlGroupState("head").q)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  7.29974202e-05
 -3.82450295e-04]
[2025-05-20 19:37:44] [10093] [error] 192.168.137.11 Get subscribe data failed with code: -302
[2025-05-20 19:37:49] [10093] [error] 192.168.137.10 Get subscribe data failed with code: -302
[2025-05-20 19:37:56] [10093] [error] 192.168.137.10 Get subscribe data failed with code: -302


In [ ]:
# 去使能机器人电机
hardware.disableRobot()